## word embedding
中文word embedding有字向量、词向量、N-GRAM词向量等。
字向量较简单，且不存在分词错误问题，较为常用，但损失了词级别信息。
词向量表达能力较强，但较为稀疏，存在分词错误问题。
1. 搜集语料，自训练：在专用领域，信息准确，效果好。
2. 大公司或机构开源：基于海量公共数据训练，信息丰富，泛化能力强。

腾讯: https://ai.tencent.com/ailab/nlp/en/embedding.html   
Facebook: https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md

## 自训练字向量
### 1. 搜集语料
用来训练的语料不限于已标注的语料，与当前任务相关的文本都可以用来构建语料库。  
也就是语料中内容的分布与任务中文本一致。   
这里我们使用已有的数据集中的所有数据作为语料，介绍如何自训练字向量。  

In [1]:
import os
import re
import codecs
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 读取全部语料
unlabeled_df = pd.read_csv('./processed_data/unlabeled.csv')
unlabeled_df.head()

,text
0,写在年末冬初孩子流感的第五天，我们仍然没有忘记热情拥抱这2020年的第一天。带着一丝迷信，早...
1,年大模型…累到以为自己 烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#
2,邱晨这就是我爹，爹， 烧快好，毕竟美好的假期拿来养病不太好，假期还是要好好享受快乐，爹，新...
3,新年的第一天感冒又 烧的也太衰了但是我要想着明天一定会好的
4,问：我们意念里有坏的想法了，天神就会给记下来，那如果有好的想法也会被记下来吗？答：那当然了。...


In [3]:
# 数据清洗，去除一些无用的字符
def preprocess_text(text):
    text = text.lower()                                     # 转为小写
    text = re.sub('[^a-z^0-9^\u4e00-\u9fa5]', '', text)     # 去除标点，这里只训练中英文字符
    text = re.sub('[0-9]', '0', text)                       # 将所有数字都转为0
    text = ' '.join(text)
    text = re.sub('\s{2,}', ' ', text)                      #合并连续的空格
    return text.strip() 

unlabeled_df['processed_text'] = unlabeled_df['text'].apply(preprocess_text)
total_sentences = unlabeled_df['processed_text']
len(total_sentences)

815846

In [4]:
# 制作训练语料，每个字符以空格分开，如果是训练词向量，则需要分词，同样每个词以空格分开
total_sentences[:3]

0    写 在 年 末 冬 初 孩 子 流 感 的 第 五 天 我 们 仍 然 没 有 忘 记 热 ...
1    年 大 模 型 累 到 以 为 自 己 烧 了 腰 疼 膝 盖 疼 腿 疼 胳 膊 疼 脖 ...
2    邱 晨 这 就 是 我 爹 爹 烧 快 好 毕 竟 美 好 的 假 期 拿 来 养 病 不 ...
Name: processed_text, dtype: object

In [5]:
# 保存处理好的语料
corpus_path = './word2vec/courpus.txt'
with codecs.open(corpus_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(total_sentences))

### 2 训练模型
使用gensim训练word2vec，参考https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
   
- sentences：可以是一个list，对于大语料集，建议使用BrownCorpus,Text8Corpus或LineSentence构建； 
- corpus_file：LineSentence格式的语料文件，训练可加速
- vector_size：是指特征向量的维度，默认为100。大的size需要更多的训练数据,但是效果会更好. 推荐值为几十到几百；   
- window：表示上下文窗口大小，当前词与预测词的最远距离；  
- min_count: 可以对字典做截断. 词频少于min_count次数的单词会被丢弃掉, 默认值为5；   
- workers: 参数控制训练的并行数，多核机器；   
- sg： 用于设置训练算法，默认为0，对应CBOW算法；sg=1则采用skip-gram算法； 
- hs: 如果为1则会采用hierarchical softmax技巧。如果设置为0（defaut），则negative sampling会被使用；   
- negative: 如果>0,则会采用negativesamping，用于设置多少个noise words；   
- epochs： 迭代次数，默认为5；   
- cbow_mean: 如果为0，则采用上下文词向量的和，如果为1（defaut）则采用均值。只有使用CBOW的时候才起作用；   
- alpha: 学习速率； 
- min_alpha: 随着训练的进行，学习率将线性下降到最小；   
- seed：用于随机数发生器。与初始化词向量有关；   
- max_vocab_size: 设置词向量构建期间的RAM限制。如果所有独立单词个数超过这个，则就消除掉其中最不频繁的一个。每一千万个单词需要大约1GB的RAM。设置成None则没有限制；   
- sample: 高频词汇的随机降采样的配置阈值，默认为1e-3，范围是(0,1e-5)；   
- hashfxn： hash函数来初始化权重。默认使用python的hash函数；   
- trim_rule： 用于设置词汇表的整理规则，指定那些单词要留下，哪些要被删除。可以设置为None（min_count会被使用）或者一个接受()并返回RU·E_DISCARD,uti·s.RU·E_KEEP或者uti·s.RU·E_DEFAU·T的函数；  
- sorted_vocab： 如果为1（defaut），则在分配word index 的时候会先对单词基于频率降序排序；  
- batch_words：每一批的传递给线程的单词的数量，默认为10000。  

![jupyter](./imgs/word2vec.png)

In [6]:
# 读取处理好的语料
with codecs.open(corpus_path, 'r', encoding='utf-8') as f:
    sentences = [line.split(' ') for line in f.read().split('\n')]
print(sentences[0])

['写', '在', '年', '末', '冬', '初', '孩', '子', '流', '感', '的', '第', '五', '天', '我', '们', '仍', '然', '没', '有', '忘', '记', '热', '情', '拥', '抱', '这', '0', '0', '0', '0', '年', '的', '第', '一', '天', '带', '着', '一', '丝', '迷', '信', '早', '晨', '给', '孩', '子', '穿', '上', '红', '色', '的', '羽', '绒', '服', '羽', '绒', '裤', '祈', '祷', '新', '的', '一', '年', '孩', '子', '们', '身', '体', '康', '健', '仍', '然', '会', '有', '一', '丝', '焦', '虑', '焦', '虑', '我', '的', '孩', '子', '为', '什', '么', '会', '过', '早', '的', '懂', '事', '从', '两', '岁', '多', '始', '关', '注', '我', '的', '情', '绪', '会', '深', '沉', '地', '说', '妈', '妈', '你', '终', '于', '笑', '了', '这', '句', '话', '像', '刀', '子', '一', '样', '扎', '入', '我']


In [7]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(sentences,   # 语料集
                 sg=1,        # 使用skip-gram算法
                 vector_size=100,  # 词向量长度
                 window=5,   # 上下问窗口大小
                 min_count=10, # 词最少出现的次数
                 workers=4,  # 训练的并行数，这里为4核
                 epochs=3    # 迭代次数，这里做演示只设置了3次
                )

In [9]:
# 保存字向量

embedding_path = './word2vec/sg_ns_100.txt'

model.wv.save_word2vec_format(embedding_path,binary=False)

### 3. 评价字向量
1.词向量的语言学特性常见的有： 
相似度评价指标，看看空间距离近的词，跟人的直觉是否一致。  
类比问题 king-queen=man-woman  
Baroni et al.的文章介绍了8种这类指标：
http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf

2.对实际NLP任务的贡献对于一些传统方法做的任务，可以直接当作特征加进去，看看提升的效果。对于用神经网络做的，可以用词向量作为词那一层的初始值，初始值选得好，就当做词向量好。 

In [8]:
# 探索字向量，查看最相似的字
for word in ['肺', '疫', '好', '老', '男']:
    result = model.wv.most_similar(word)
    print(word)
    print(result)

肺
[('炎', 0.7689979672431946), ('冠', 0.657269299030304), ('型', 0.6283558011054993), ('状', 0.5542781949043274), ('v', 0.5362928509712219), ('病', 0.5229945778846741), ('新', 0.5206800103187561), ('疫', 0.47544923424720764), ('情', 0.4736940562725067), ('o', 0.45468762516975403)]
疫
[('情', 0.712246835231781), ('击', 0.6222784519195557), ('控', 0.6051361560821533), ('役', 0.5152427554130554), ('阻', 0.49533212184906006), ('炎', 0.48094266653060913), ('战', 0.4796537458896637), ('蔓', 0.4768461585044861), ('肺', 0.47544926404953003), ('型', 0.47047603130340576)]
好
[('哦', 0.6496912240982056), ('呀', 0.639189600944519), ('做', 0.6037340760231018), ('喔', 0.5598272681236267), ('也', 0.5462799668312073), ('哎', 0.5362486243247986), ('己', 0.5333588719367981), ('很', 0.5286406874656677), ('哇', 0.5281414985656738), ('吧', 0.5241935849189758)]
老
[('爷', 0.6205733418464661), ('姓', 0.5926575064659119), ('婆', 0.58570796251297), ('爹', 0.5391449332237244), ('爸', 0.5328716039657593), ('师', 0.5264418721199036), ('父', 0.5206684